In [ ]:
# tous les imports
import numpy as np
import re
import matplotlib.pyplot as plt
import requests
import pickle as pkl
import time

import datetime

# Compréhension du format de l'API blablacar

In [ ]:
key = ... # demander une clé sur le site et la mettre ici

req = "https://public-api.blablacar.com/api/v2/trips?key="+key+"&fn=Paris&limit=100"
dataStation = requests.get(req)

len(dataStation.json())

# dict_keys(['links', 'pager', 'trips', 'top_trips', 'facets', 'lowest_price', 'lowest_price_object', 
#'full_trips_count', 'total_trip_count_to_display', 'sorting_algorithm', 'has_bus'])

print(dataStation.json()['pager']['pages'])
#
u = dataStation.json()['trips']
print(len(u))
print(u[0])
#for u in dataStation.json():
#    print

# Collecte des données
Une fois le format de l'API à peu près cerné, on lance la collecte des données sur 7 villes

In [ ]:
# formulation des requetes:
villes = ['Paris', 'Marseille', 'Grenoble', 'Lille', 'Strasbourg', 'Nantes', 'Bordeaux']

# il faut mettre une cle valide à la place de KEY
urlbase = "https://public-api.blablacar.com/api/v2/trips?"

# clé déjà renseignée plus haut
data = []
cpt = 0
for v in villes:
    # requête fantome pour récupérer le nombre de pages de résultats
    req = urlbase+"key="+key+"&fn="+v+"&limit=100"
    dataStation = requests.get(req)
    npages = dataStation.json()['pager']['pages']
    print("Ville = " + v + " npages = " + str(npages))
    for p in range(npages):
            cpt += 1
            req = urlbase+"key="+key+"&fn="+v+"&limit=100&page="+str(p+1)
            print(req)
            dataStation = requests.get(req)
            tmp = dataStation.json()
            data.append(tmp)
            time.sleep(1)
            #print(tmp)

# sauvegarde brute
pkl.dump(data, open('blablacar.pkl', 'wb'))

print(cpt)  

In [ ]:
# Fonction pour aplatir un json
# NB: panda fait ça très bien... Mais c'est nul panda :)
def applatir(d):
    res = dict()
    for k,v in d.items():
        if type(v) == dict:
            tmp = applatir(v)
            for kk,vv in tmp.items():
                res[k+"."+kk]=vv
        else: res[k]=v
    return res

In [ ]:
# traitement/filtrage des données récoltées

# liste des clés intéressantes pour moi:
keys = ['departure_date',\
        'departure_place.city_name', 'departure_place.latitude', 'departure_place.longitude',\
        'departure_meeting_point.city_name','departure_meeting_point.latitude', 'departure_meeting_point.longitude',
        'arrival_place.city_name', 'arrival_place.latitude', 'arrival_place.longitude', 'price.value',\
       'car.make', 'car.comfort_nb_star', 'distance.value']

d = pkl.load(open('blablacar.pkl', 'rb'))

ppdb = []

for r in d:
    trips = r['trips']
    for t in trips:
        ft = applatir(t)
        trips_red = [ft.get(k,'unknown') for k in keys]
        ppdb.append(trips_red)
print(d[0])

In [ ]:
# traitement phase 2: reécupération des dates, indexation des villes...

ppdba = np.array(ppdb)
villes = set(ppdba[:,1]).union(set(ppdba[:,4])).union(set(ppdba[:,7]))
villes = dict({(key,idx) for idx, key in enumerate(villes)})
marques = dict({(key,idx) for idx, key in enumerate(set(ppdba[:,11]))})
pp2db = []

for i,t in enumerate(ppdb):
    date_time_obj = datetime.datetime.strptime(t[0],'%d/%m/%Y %H:%M:%S')
    tmp = [date_time_obj.year, date_time_obj.month, date_time_obj.day, date_time_obj.hour]
    tmp +=  [villes[t[1]], villes[t[7]]]
    tmp +=  t[2:4]
    #tmp +=  t[5:7]
    tmp += t[8:11]
    tmp += [marques[t[-3]]]
    if t[-2] == 'unknown':
        tmp += [-1]
    else:
        tmp += [t[-2]]
    tmp += [t[-1]]
    pp2db.append(tmp)
    #if i==5:
    #    break
print(date_time_obj.year)
print(marques)
pp2db = np.array(pp2db)
#print(pp2db)

In [ ]:
print(ppdb[0])
print(pp2db)

In [ ]:
cols = ['annee', 'mois', 'jour', 'heure', 'dep_ville' ,'arr_ville' ,'dep_coord_x','dep_coord_y',\
        'arr_coord_x','arr_coord_y', 'prix', 'marque', 'stars_confort', 'distance']

pkl.dump({'indexcol': cols , 'data':pp2db, 'villes': villes, 'marques':marques }, open('blablacar_pp.pkl', 'wb'))